In [1]:
#Import Libraries
import pandas as pd
import missingno as msn
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [5]:
#Read data file obtained from datos.gov.co
# path = "C:/Users/User/Downloads/ds4a-secopII/" #path Carlos
path = "C:/Users/VMNR/Desktop/" #path vicman
# path = "../data/" #path GENERAL
file_name = 'SECOP_Electronicos_Cleaned.csv'

#Select the columns that we want to parse as dates.
date_cols = ['Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato']

df = pd.read_csv(path+file_name, parse_dates=date_cols, low_memory=False)


In [6]:

print('The shape of the original dataset is : ' + str(df.shape))

The shape of the original dataset is : (1562161, 35)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562161 entries, 0 to 1562160
Data columns (total 35 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Unnamed: 0                       1562161 non-null  int64         
 1   Nombre Entidad                   1562161 non-null  object        
 2   Departamento                     1562161 non-null  object        
 3   Orden                            1562161 non-null  object        
 4   Sector                           1562161 non-null  object        
 5   Rama                             1562161 non-null  object        
 6   Entidad Centralizada             1562161 non-null  object        
 7   Estado Contrato                  1562161 non-null  object        
 8   Tipo de Contrato                 1562161 non-null  object        
 9   Modalidad de Contratacion        1562161 non-null  object        
 10  Fecha de Inicio del Contrato  

In [10]:
cat_cols = ['Departamento', 'Orden', 'Sector', 'Rama','Entidad Centralizada', 'Estado Contrato', 'Tipo de Contrato', 'Modalidad de Contratacion',
            'Es Grupo','Es Pyme', 'Destino Gasto', 'EsPostConflicto', 'Delay']
cat_values = {key:'category' for key in cat_cols}
df = df.astype(cat_values)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562161 entries, 0 to 1562160
Data columns (total 35 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Unnamed: 0                       1562161 non-null  int64         
 1   Nombre Entidad                   1562161 non-null  object        
 2   Departamento                     1562161 non-null  category      
 3   Orden                            1562161 non-null  category      
 4   Sector                           1562161 non-null  category      
 5   Rama                             1562161 non-null  category      
 6   Entidad Centralizada             1562161 non-null  category      
 7   Estado Contrato                  1562161 non-null  category      
 8   Tipo de Contrato                 1562161 non-null  category      
 9   Modalidad de Contratacion        1562161 non-null  category      
 10  Fecha de Inicio del Contrato  

In [12]:
df2 = df[df['Dias Adicionados']>0]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231621 entries, 17 to 1562154
Data columns (total 35 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Unnamed: 0                       231621 non-null  int64         
 1   Nombre Entidad                   231621 non-null  object        
 2   Departamento                     231621 non-null  category      
 3   Orden                            231621 non-null  category      
 4   Sector                           231621 non-null  category      
 5   Rama                             231621 non-null  category      
 6   Entidad Centralizada             231621 non-null  category      
 7   Estado Contrato                  231621 non-null  category      
 8   Tipo de Contrato                 231621 non-null  category      
 9   Modalidad de Contratacion        231621 non-null  category      
 10  Fecha de Inicio del Contrato     231621 no

In [ ]:
df2